In [30]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import math
from scipy.spatial import distance


# Number of images (we have 4 in this example)
N = 4

# Read image
im_dictionary = {}

# The names of the example images are iterable
for i in range(0,N):
    im_dictionary["Image{0}".format(i)] = cv2.imread("Example_Pictures/IMG_"+str(8105+i)+".jpg")

In [31]:
# Because the camera does not stand still, we want fixed reference points to find
# out if the laser point has moved
# We are using the white dots in the background for this

# The color range we want the blobs to be in
lower = (70,70,70)
upper = (130,130,2130)

mask_dictionary = {}
for i in range(0,N):
    mask_dictionary["Mask{0}".format(i)] =  cv2.dilate(cv2.inRange(im_dictionary["Image{0}".format(i)], lower, upper), None, iterations=1)


# Setup SimpleBlobDetector parameters
params = cv2.SimpleBlobDetector_Params()


# Change thresholds
params.minThreshold = 0;
params.maxThreshold = 256;

# Only keep blobs in this size range
params.filterByArea = True
params.minArea = 3500
params.maxArea = 40000


# Detect blobs
blob = cv2.SimpleBlobDetector_create(params)

# save keypoints
keypoint_dictionary = {}
for i in range(0,N):
    keypoint_dictionary["Keypoints{0}".format(i)] = blob.detect(255-mask_dictionary["Mask{0}".format(i)])

In [32]:
background_coordinates_dictionary = {}
for i in range(0,N):
    background_coordinates_dictionary["Coordinates{0}".format(i)] =  [keypoint.pt for keypoint in keypoint_dictionary["Keypoints{0}".format(i)]]

In [33]:
# Laserpoint detection
lower = (200,110,160)
upper = (255,255,255)

mask_dictionary = {}
for i in range(0,N):
    mask_dictionary["Mask{0}".format(i)] = cv2.dilate(cv2.inRange(im_dictionary["Image{0}".format(i)], lower, upper), None, iterations=1)

# Setup SimpleBlobDetector parameters
params = cv2.SimpleBlobDetector_Params()


# Change thresholds
params.minThreshold = 0;
params.maxThreshold = 256;


params.filterByArea = True
params.minArea = 600
params.maxArea = 40000


params.filterByConvexity = True
params.minConvexity = 0.5


# Detect blobs
blob = cv2.SimpleBlobDetector_create(params)



laserpoint_dictionary = {}
for i in range(0,N):
    laserpoint_dictionary["Keypoints{0}".format(i)] = blob.detect(255-mask_dictionary["Mask{0}".format(i)])


In [38]:
laser_coordinates_dictionary = {}
for i in range(0,N):
    laser_coordinates_dictionary["Coordinates{0}".format(i)] =  [keypoint.pt for keypoint in laserpoint_dictionary["Keypoints{0}".format(i)]]

In [39]:
# We want to compare the distance of the laser point to just one point
reference_point_dictionary = {}

for i in range(0,N):
    reference_point_dictionary["Reference_Point{0}".format(i)] = min(background_coordinates_dictionary["Coordinates{0}".format(i)], key=lambda coordinate: (laser_coordinates_dictionary["Coordinates0"][0][0]-coordinate[0])**2+(laser_coordinates_dictionary["Coordinates0"][0][1]-coordinate[1])**2)

In [40]:
distance_dictionary = {}

for i in range(0,N):
    distance_dictionary["Distances{0}".format(i)] = [laser_coordinates_dictionary["Coordinates{0}".format(i)][0][0]-reference_point_dictionary["Reference_Point{0}".format(i)][0], laser_coordinates_dictionary["Coordinates{0}".format(i)][0][1]-reference_point_dictionary["Reference_Point{0}".format(i)][1]]

In [41]:
width = len(mask_dictionary["Mask0"][0])
height = len(mask_dictionary["Mask0"])

print("Images have a size of "+str(width)+" x "+str(height))

Images have a size of 3456 x 2304


In [42]:
# We take the first image as reference image
reference_image_distance = distance_dictionary["Distances0"]

# This is how the laserpoint has moved
final_dictionary = {}

for i in range(0,N):
    final_dictionary["Movement{0}".format(i)] =  distance.euclidean(np.array(distance_dictionary["Distances{0}".format(i)]),np.array(reference_image_distance))

In [43]:
"The distance between two background points is 18mm."

'The distance between two background points is 18mm.'

In [46]:
point_list = background_coordinates_dictionary['Coordinates0']

reference_point = point_list[15]
# We delete the 15th point out of our list of white points to get the closest point to our reference point
del point_list[15]

closest_point = min(point_list, key=lambda coordinate: (reference_point[0]-coordinate[0])**2+(reference_point[1]-coordinate[1])**2)
pixel_distance = np.linalg.norm(closest_point)-np.linalg.norm(reference_point)
"The distance between two background points is "+str(pixel_distance)+" pixels on our image"

'The distance between two background points is 281.2403582409454 pixels on our image'

In [47]:
factor = 18/pixel_distance
"We scale down our movements by the factor "+str(factor)+"."

'We scale down our movements by the factor 0.0640021941110563.'

In [58]:
"There were always two pictures taken. The error that occurs just through the inaccuracy of the program and the movement of the camera is about "+str(round(final_dictionary["Movement1"]*factor, 3))+"mm."

'There were always two pictures taken. The error that occurs just through the inaccuracy of the program and the movement of the camera is about 0.012mm.'

In [52]:
for i in range(2,N):
    print("The laser point has moved "+str(round((final_dictionary["Movement{0}".format(i)])*factor,3))+"mm in image "+str(i+1)+".")
          
print("Remember there is a small inaccuracy caused by the program.")

The laser point has moved 0.038mm in image 3.
The laser point has moved 0.039mm in image 4.
Remember there is a small inaccuracy caused by the program.


In [53]:
for i in range(0,N):
    print("The distance between the laser point and the reference point is "+str(round(math.sqrt(distance_dictionary["Distances{0}".format(i)][0]**2+distance_dictionary["Distances{0}".format(i)][1])*factor,3))+"mm.")

The distance between the laser point and the reference point is 2.854mm.
The distance between the laser point and the reference point is 2.857mm.
The distance between the laser point and the reference point is 2.851mm.
The distance between the laser point and the reference point is 2.872mm.


In [54]:
# printing out an example image
keypoints = np.concatenate((keypoint_dictionary["Keypoints0"], laserpoint_dictionary["Keypoints0"]))

In [56]:
reversed_mask = 255-mask_dictionary["Mask0"]

cv2.namedWindow("output", cv2.WINDOW_NORMAL)        # Create window with freedom of dimensions
img_with_keypoints = cv2.drawKeypoints(reversed_mask, keypoints, None,color=(0,255,0), flags =cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
img_with_keypoints = cv2.resize(img_with_keypoints, (960, 540)) # Resize image

# Show blobs
cv2.imshow("Keypoints", img_with_keypoints)
cv2.waitKey(0)

120

In [57]:
cv2.destroyAllWindows()